# Getting Started Code for [Chunin Exams Food Track- CV'2021](https://www.aicrowd.com/challenges/chunin-exams-food-track-cv-2021) on AIcrowd
#### Author : Pulkit Gera

## Download Necessary Packages 📚

In [ ]:
!pip install aicrowd-cli

     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 163kB 7.6MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 204kB 12.6MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


## Download Data
The first step is to download out train test data. We will be training a model on the train data and make predictions on test data. We submit our predictions.


In [ ]:
API_KEY = "fd1aed7204268f0bae2780fb808a4063" #Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [ ]:
!aicrowd dataset download --challenge chunin-exams-food-track-cv-2021

train_images.zip: 100% 754M/754M [00:08<00:00, 90.5MB/s]
test_images.zip: 100% 33.9M/33.9M [00:00<00:00, 65.8MB/s]
train.csv: 100% 253k/253k [00:00<00:00, 3.07MB/s]
test.csv: 100% 7.27k/7.27k [00:00<00:00, 913kB/s]


In [ ]:
!unzip train_images.zip
!unzip test_images.zip

#Libraries

In [ ]:
import cv2
import os
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import numpy as np

import keras
import glob
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from mlxtend.preprocessing import minmax_scaling
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical
from keras.layers import AveragePooling2D
from keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input, BatchNormalization, Multiply, Activation
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras import backend as K
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
import tensorflow as tf

from PIL import Image, ImageOps

#Load data

In [ ]:
def load_images_from_folder(folder):

  train_data=pd.read_csv('train.csv')
  images_names=train_data['ImageId']

  X=[]
  for img_name in images_names:
    img = Image.open('train_images/'+img_name)
    X.append(img.convert('RGB').resize((128, 128), Image.ANTIALIAS))

  return X
  
def load_labels(folder):
    with open(folder, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        labels = []  
        line_count = 0  
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            labels.append(row['ClassName'])
            line_count += 1
        
        return labels
    

train_images = load_images_from_folder('./train.csv')
train_labels = load_labels('./train.csv')
test_images = load_images_from_folder('./test_images/')
train_images = np.array([np.array(x) for x in train_images])
train_images = train_images/255.


In [ ]:
le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = train_labels_enc.reshape(len(train_labels_enc), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

train_labels_enc = le.transform(train_labels)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = train_labels_enc.reshape(len(train_labels_enc), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

train_labels = onehot_encoded 

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.15, random_state=42)
# X_train, y_train = train_images, train_labels

#For Part 2

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (160, 160, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')
for layer in pre_trained_model.layers:
  layer.trainable = False

from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(61, activation='softmax')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['categorical_accuracy'])


train_datagen = ImageDataGenerator(
    rotation_range=40,      #rotation
    width_shift_range=0.2,  #translation with widht
    height_shift_range=0.2,  #translation with height
    brightness_range=[0.2,1.0] #color brighness change
    )

model.fit(X_train, y_train, epochs = 30, batch_size = 32, validation_split=0.15)
# model.fit_generator(train_datagen.flow(X_train, y_train, batch_size = 32), validation_data = (X_val,y_val), steps_per_epoch = X_train.shape[0]/32,  epochs = 25)

In [ ]:
model.save('mymodel')

INFO:tensorflow:Assets written to: mymodel/assets


In [ ]:
train_data=pd.read_csv('test.csv')
images_names=train_data['ImageId']

X=[]
for img_name in images_names:
  img = Image.open('test_images/'+img_name)
  X.append(img.convert('RGB').resize((160, 160), Image.ANTIALIAS))


X = np.array([np.array(x) for x in X])
X = X / 255.
ans = model.predict(X)

res = []
for i in range(ans.shape[0]):
  ind = np.argmax(ans[i])
  res.append(ind)
res = (le.inverse_transform(res))



In [ ]:
x= np.dstack((np.arange(0, res.size),res))[0]
np.savetxt("submission.csv",x, fmt='%s', delimiter=",",header=",ClassName")

In [ ]:
model.save('mymodel')

INFO:tensorflow:Assets written to: mymodel/assets
